In [1]:
%matplotlib inline
from astropy.io import fits
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Let's import Astroquery
from astroquery.mast import Observations

# AND also Astroquery Catalogs!
from astroquery.mast import Catalogs

## Catalogs

The list of catalogs (and thus parameters) are:

- The Hubble Source Catalog (**HSC**)
- The GALEX Catalog (**V2** and **V3**)
- The Gaia (**DR1** and **DR2**) and **TGAS** Catalogs
- The TESS Input Catalog (**TIC**)
- The TESS Candidate Target List (**CTL**)
- The Disk Detective Catalog
- PanSTARRS (**DR1**, **DR2**)

Let's use the TESS catalog (**TIC**)

Following the examples given in the 

In [4]:
catalog_data.colnames

['ID',
 'ra',
 'dec',
 'pmRA',
 'pmDEC',
 'Tmag',
 'objType',
 'typeSrc',
 'version',
 'HIP',
 'TYC',
 'UCAC',
 'TWOMASS',
 'SDSS',
 'ALLWISE',
 'GAIA',
 'APASS',
 'KIC',
 'POSflag',
 'e_pmRA',
 'e_pmDEC',
 'PMflag',
 'plx',
 'e_plx',
 'PARflag',
 'gallong',
 'gallat',
 'eclong',
 'eclat',
 'Bmag',
 'e_Bmag',
 'Vmag',
 'e_Vmag',
 'umag',
 'e_umag',
 'gmag',
 'e_gmag',
 'rmag',
 'e_rmag',
 'imag',
 'e_imag',
 'zmag',
 'e_zmag',
 'Jmag',
 'e_Jmag',
 'Hmag',
 'e_Hmag',
 'Kmag',
 'e_Kmag',
 'TWOMflag',
 'prox',
 'w1mag',
 'e_w1mag',
 'w2mag',
 'e_w2mag',
 'w3mag',
 'e_w3mag',
 'w4mag',
 'e_w4mag',
 'GAIAmag',
 'e_GAIAmag',
 'e_Tmag',
 'TESSflag',
 'SPFlag',
 'Teff',
 'e_Teff',
 'logg',
 'e_logg',
 'MH',
 'e_MH',
 'rad',
 'e_rad',
 'mass',
 'e_mass',
 'rho',
 'e_rho',
 'lumclass',
 'lum',
 'e_lum',
 'd',
 'e_d',
 'ebv',
 'e_ebv',
 'numcont',
 'contratio',
 'disposition',
 'duplicate_id',
 'priority',
 'eneg_EBV',
 'epos_EBV',
 'EBVflag',
 'eneg_Mass',
 'epos_Mass',
 'eneg_Rad',
 'epos_Rad',

In [5]:
catalog_data = Catalogs.query_object("158.47924 -7.30962", catalog="TIC")
print(catalog_data[:10])

    ID           ra               dec        ... wdflag     dstArcSec     
--------- ---------------- ----------------- ... ------ ------------------
841736289 158.475246467012 -7.29984176473098 ...      0  37.98031847255667
841736281 158.483019303286 -7.32320013067735 ...      0  50.71676154870489
 56661355 158.467833401313 -7.31994230664877 ...      0 55.134249824524694
 56661360 158.460687211256 -7.29614097021537 ...      0  82.11855685017336
 56662075 158.502417380993 -7.31732945908738 ...      0  87.28954844177136
841736294 158.455174599494 -7.29642616731829 ...      1  98.18583423560875
841736295 158.461088415216  -7.2875177250041 ...      1 102.62671663330786
841736277 158.469287160216 -7.33637889451687 ...      0 102.67816386866188
841736296 158.470331723001  -7.2816687605798 ...      0 105.53278544229352
 56661356  158.44887619493 -7.30920784493643 ...      0 108.43155086927304


So in the below example, I still need to hard-code the TIC-ID in, and perhaps this will/should be changed in the future.

In [14]:
ticid = 261105201

starName = "TIC " + str(ticid)
radSearch = 4/60 #radius in degrees // arcmins

catalogData = Catalogs.query_object(starName, radius = radSearch, catalog = "TIC")
ra = catalogData[0]['ra']
dec = catalogData[0]['dec']

# Print out the first row in the table
print( catalogData[:5]['ID', 'Tmag', 'Jmag', 'ra', 'dec', 'objType'] )

    ID      Tmag   Jmag         ra               dec        objType
--------- ------- ------ ---------------- ----------------- -------
261105201  8.3629   7.74 82.8273670408244 -79.0087723001529    STAR
724151530 18.7511    nan 82.8150127457216 -79.0132058191133    STAR
261105202 15.6838 13.738  82.807947620659 -79.0136350375361    STAR
724151528 20.1425    nan   82.79364170498 -79.0085739998184    STAR
724151541 19.6238    nan 82.8606445683429 -79.0110416543022    STAR


In [15]:
obsCount = Observations.query_criteria_count(obs_collection = "TESS",
                                             dataproduct_type = ["timeseries"],
                                             sequence_number = 1,
                                             proposal_id = "*G011183*") # related to particular investigators data
print("Number of Observations: %i" % obsCount)

Number of Observations: 42


In [16]:
obsTable = Observations.query_criteria(obs_collection = "TESS",
                                       dataproduct_type = ["timeseries"],
                                       sequence_number = 1,
                                       proposal_id = "*G011183*")
obsTable[0:5]['obsid','proposal_id','obs_id']

obsid,proposal_id,obs_id
str11,str31,str47
17000009457,G011112_G011183_G011132,tess2018206045859-s0001-0000000038846515-0120-s
17000006704,G011183_G011025_G011048_G011250,tess2018206045859-s0001-0000000231836962-0120-s
17000011124,G011183_G011250,tess2018206045859-s0001-0000000317057430-0120-s
17000003763,G011183_G011025_G011048_G011250,tess2018206045859-s0001-0000000099734092-0120-s
17000001820,G011155_G011183_G011184_G011048,tess2018206045859-s0001-0000000129649472-0120-s


In [18]:
dataProducts = Observations.get_product_list(obsTable[0:6])
dataProducts.colnames

['obsID',
 'obs_collection',
 'dataproduct_type',
 'obs_id',
 'description',
 'type',
 'dataURI',
 'productType',
 'productGroupDescription',
 'productSubGroupDescription',
 'productDocumentationURL',
 'project',
 'prvversion',
 'proposal_id',
 'productFilename',
 'size',
 'parent_obsid',
 'dataRights']

This gives us the column names in the "dataProducts" observation list. For the purposes of learning how this all works, we only need (for now) to worry about object ID, filename, and description (to get the object id reference number; have access to the filename; and see what the file contains, respectively).

In [19]:
shortList = dataProducts['obsID', 'productFilename', 'description']
shortList

obsID,productFilename,description
str11,str63,str27
17000000913,tess2018206190142-s0001-s0001-0000000025155310-00106_dvr.pdf,full data validation report
17000000913,tess2018206190142-s0001-s0001-0000000025155310-00106_dvr.xml,full data validation report
17000000913,tess2018206190142-s0001-s0001-0000000025155310-00106_dvm.pdf,Data validation mini report
17000000913,tess2018206190142-s0001-s0001-0000000025155310-01-00106_dvs.pdf,TCE summary report
17000000913,tess2018206190142-s0001-s0001-0000000025155310-00106_dvt.fits,Data validation time series
17000000913,tess2018206045859-s0001-0000000025155310-0120-s_lc.fits,Light curves
17000000913,tess2018206045859-s0001-0000000025155310-0120-s_tp.fits,Target pixel files
17000009457,tess2018206190142-s0001-s0001-0000000038846515-00106_dvr.pdf,full data validation report
17000009457,tess2018206190142-s0001-s0001-0000000038846515-00106_dvr.xml,full data validation report


In [24]:
# This is the part that breaks

#want = dataProducts['description'] == "Light curves"

#type(dataProducts['description'])

#print(dataProducts[want])

astropy.table.column.MaskedColumn

In [25]:
#
# Let's do this more (BUT NOT AS) Pythonic, eh?
# Reminder for self: This is (essentially) how Pythonic lists work:
#
#[ 'expression' for 'item' in 'list' if 'conditional' ]
# ==
#for item in list:
#    if conditional:
#        expression
#
fits_list = [i['productFilename'] for i in shortList if i['description'] == "Light curves"]
pixl_list = [i['productFilename'] for i in shortList if i['description'] == "Target pixel files"]
fits_list

['tess2018206045859-s0001-0000000025155310-0120-s_lc.fits',
 'tess2018206045859-s0001-0000000038846515-0120-s_lc.fits',
 'tess2018206045859-s0001-0000000099734092-0120-s_lc.fits',
 'tess2018206045859-s0001-0000000129649472-0120-s_lc.fits',
 'tess2018206045859-s0001-0000000231836962-0120-s_lc.fits',
 'tess2018206045859-s0001-0000000317057430-0120-s_lc.fits']

#### AWW YEAH AUTO GET FILES WOOOOO

I guess I didn't need to do the above tbh? It's nice to have though. In order to download files I still need the original list, in my case "dataProducts", but yeah. I'm glad I worked out the above anyways.

In [41]:
manifest = Observations.download_products(dataProducts, extension="fits")
#print(manifest)

INFO: Found cached file ./mastDownload/TESS/tess2018206045859-s0001-0000000139754153-0120-s/tess2018206045859-s0001-0000000139754153-0120-s_lc.fits with expected size 2039040. [astroquery.query]
INFO: Found cached file ./mastDownload/TESS/tess2018206045859-s0001-0000000139754153-0120-s/tess2018206045859-s0001-0000000139754153-0120-s_tp.fits with expected size 58017600. [astroquery.query]


Reproduce analysis on LC to see if exo - if so how big / period?
I have data now I need to use it
Pick region whewre I KNOW there are exo, grab a bunch, get LCs, plot, build ML:
- Does LC show existence of exo
- IF there is one, get info (period, radius, etc)
Go Fast and Break Things! (be the hare, not the tortoise)

More work on these tools should turn into background info, ie, text for thesis

Work out which ML algo is gonna be useful / beneficial to my investigation